In [43]:
import random
import numpy as np
def generate_scp_dataset(num_universe_items, num_sets, min_set_size, max_set_size):
    """
    Generate a dataset for the Set Covering Problem.

    :param num_universe_items: Number of items in the universe.
    :param num_sets: Number of sets to generate.
    :param min_set_size: Minimum number of items in each set.
    :param max_set_size: Maximum number of items in each set.
    :return: A tuple (universe, x) where universe is a set of all items,
             and x is a binary matrix representing the sets.
    """
    # Generate the universe of items
    universe = set(range(num_universe_items))

    # Initialize the binary matrix for sets
    x = np.zeros((num_sets, num_universe_items), dtype=int)

    # Generate the sets and update the binary matrix
    for i in range(num_sets):
        set_size = random.randint(min_set_size, max_set_size)
        set_items = random.sample(list(universe), set_size)  # Convert the set to a list here
        for item in set_items:
            x[i, item] = 1

    # Ensure every item is covered at least once
    uncovered_items = universe.copy()
    for item in universe:
        if not np.any(x[:, item]):
            random_set = random.randint(0, num_sets - 1)
            x[random_set, item] = 1
            uncovered_items.discard(item)

    # If there are still uncovered items, add them to random sets
    while uncovered_items:
        item = uncovered_items.pop()
        random_set = random.randint(0, num_sets - 1)
        x[random_set, item] = 1

    return x

x = generate_scp_dataset(100, 100, 1, 50)
print(x)


[[0 1 1 ... 0 0 0]
 [0 0 1 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 1 0 ... 1 0 0]
 [1 1 0 ... 0 0 0]
 [1 0 0 ... 1 1 0]]


In [3]:
# Taken from https://github.com/alisaab/l0bnb/tree/master

from scipy import optimize as sci_opt
import numpy as np


class Node:
    def __init__(self, parent, node_key, zlb: list, zub: list, model = None):
        """
        Initialize a Node

        Parameters
        ----------
        parent: Node or None
            the parent Node
        
        node_key: Str
            Name associated with Node, used for Node Lookup

        """
        
        self.parent_dual = parent.dual_value if parent else None
        self.parent_primal = parent.primal_value if parent else None

        self.level = parent.level + 1 if parent else 0
        self.zlb = zlb
        self.zub = zub
        self.z = None

        self.upper_bound = None
        self.primal_value = None
        self.dual_value = None

        self.support = None
        self.upper_z = None
        self.primal_beta = None

        # Each node will store it's children's nodes and current state
        self.node_key = node_key
        self.parent_key = None
        self.is_leaf = True
        self.left = None
        self.right = None
        self.state = None
        
        self.model = model

    def get_info(self):
        return f'node_key: {self.node_key}, is_leaf: {self.is_leaf}, ' \
            f'state: {self.state}'
    
    def assign_children(self, left_child=None, right_child=None):
        '''
        Function assigns children nodes to parent and 
        parent is set to no longer leaf

        inputs:
            left_child: Node associated with Left Child
            right_child: Node associated with Right Child
        '''
        if left_child is not None:
            self.left = left_child
            self.left.parent_key = self.node_key
            self.is_leaf = False
        
        if right_child is not None:
            self.right = right_child
            self.right.parent_key = self.node_key
            self.is_leaf = False

    def __str__(self):
        return f'level: {self.level}, lower cost: {self.primal_value}, ' \
            f'upper cost: {self.upper_bound}'

    def __repr__(self):
        return self.__str__()

    def __lt__(self, other):
        if self.level == other.level:
            if self.primal_value is None and other.primal_value:
                return True
            if other.primal_value is None and self.primal_value:
                return False
            elif not self.primal_value and not other.primal_value:
                return self.parent_primal > \
                       other.parent_cost
            return self.primal_value > other.lower_bound_value
        return self.level < other.level

In [47]:
from pyscipopt import Model, quicksum
import numpy as np
import random

class Problem:
    
    def __init__(self, x):
        self.x = x
        # Setup the model
        self.model = Model("SetCoverRelaxed")
        num_sets, num_universe_items = self.x.shape
        set_vars = [self.model.addVar(vtype="C", name=f"x_{i}") for i in range(num_sets)]
        # Constraints
        for j in range(num_universe_items):
            self.model.addCons(quicksum(self.x[i, j] * set_vars[i] for i in range(num_sets)) >= 1)
        # Objective
        self.model.setObjective(quicksum(set_vars), "minimize")
        self.model.setIntParam("display/verblevel", 1)

    def lower_solve(self, node):
        """
        Solves the relaxed set cover problem (linear programming relaxation).

        Parameters:
            node (Node): The current node in the branch and bound tree.

        Returns:
            tuple: 
                - primal_value (float): The primal value of the solution.
                - dual_value (float): The dual value of the solution.
        """
        
        # Adding branching decisions from node state (zlb and zub)
        node.model.freeTransform()
        lp_vars = [var for var in node.model.getVars()]
        for i in node.zlb:
            node.model.chgVarUb(lp_vars[i], 0)
        for i in node.zub:
            node.model.chgVarLb(lp_vars[i], 1)
        
        node.model.optimize()

        primal_value = node.model.getPrimalbound()
        dual_value = node.model.getDualbound()

        # Update the attributes of the provided Node instance
        node.primal_value = primal_value
        node.dual_value = dual_value
        node.z = [node.model.getVal(var) for var in lp_vars]
        return node.primal_value, node.dual_value
    
    def upper_solve(self, node):
        """
        Solves the set cover problem using LP rounding to provide an upper bound.

        Parameters:
            node (Node): The current node in the branch and bound tree.

        Returns:
            float: The cost of the selected sets (upper bound).
            list: Indices of selected sets.
        """
        num_sets, num_universe_items = self.x.shape
        set_costs = [1 for _ in range(len(self.x))]  # Assuming each set has a cost of 1
        threshold = 0.5
        selected_sets = [i for i, val in enumerate(node.z) if val >= threshold]

        # Ensure all items are covered
        for j in range(num_universe_items):
            if not any(self.x[i, j] for i in selected_sets):
                # Find the set with the highest fractional value that covers this item
                sets_covering_item = [i for i in range(num_sets) if self.x[i, j] > 0]
                max_contrib_set = max(sets_covering_item, key=lambda i: node.z[i])
                selected_sets.append(max_contrib_set)

        selected_sets = list(set(selected_sets))  # Remove duplicates
        upper_bound = sum(set_costs[i] for i in selected_sets)
        node.upper_bound = upper_bound
        node.upper_z = [1 if i in selected_sets else 0 for i in range(len(self.x[0]))]
        return upper_bound

In [48]:
import numpy as np
from copy import deepcopy
from node import Node
from random import choice, choices
import math
from operator import attrgetter
from settings import MAX_ITERS
from scipy import optimize as sci_opt

def reverse_lookup(d, val):
  for key in d:
    if d[key] == val:
      return key


class tree():
    def __init__(self, problem):
        self.problem = problem
        self.tree_stats = None

        # Algorithm variables - also reset with set_root below
        self.active_nodes = dict()      # Current Nodes
        self.all_nodes = dict()         # All Nodes
        self.step_counter = 0           # Number branch steps taken
        self.node_counter = 0
        self.best_int = math.inf            # Best integer solution value
        self.candidate_sol = None           # Best integer solution betas
        self.lower_bound = None             # Minimum relaxation value of all nodes
        self.best_int = math.inf 
        ## Check if needed
        self.root = None

    def start_root(self):
        # Initializes the nodes with a root node
        # Use warm start for upper bound
        # Initialize root node
        root_node = Node(parent=None, node_key='root_node',zlb=[], zub=[], model = self.problem.model)
        self.active_nodes['root_node'] = root_node
        self.all_nodes['root_node'] = root_node
        self.problem.lower_solve(root_node)
        self.problem.upper_solve(root_node)

        # Update bounds and opt gap
        self.lower_bound = root_node.primal_value
        if (root_node.upper_bound < self.best_int):
            self.best_int = root_node.upper_bound
            self.candidate_sol = root_node.upper_z
        self.initial_optimality_gap = (self.best_int - self.lower_bound) / self.lower_bound
        self.optimality_gap = self.initial_optimality_gap
        self.node_counter += 1
        self.tree_stats = self.get_tree_stats()

        # Start Tree
        self.root = self.active_nodes['root_node']

    

    def get_info(self):
        # Summary info of current state
        info = {'step_count': self.step_counter,
                'node_count': self.node_counter,
                'best_primal_value': self.best_int,
                'candidate_sol': self.candidate_sol,
                'lower_bound': self.lower_bound,
                'init_opt_gap' : self.initial_optimality_gap,
                'opt_gap': self.optimality_gap}
        return(info)

    def get_tree_stats(self):
        """
        Returns tree statistics

        Returns:
            numpy.ndarray: Tree specific statistics.
                - number of steps taken
                - number of active nodes
                - number of candidate solution variables
                - lower bound
                - best integer solution
                - initial optimality gap
                - current optimality gpa
        """
        # Returns summary statisitics that describe the set of all active nodes
        tree_stats = np.array([self.step_counter,
                               len(self.active_nodes),
                               len(self.candidate_sol),
                               self.lower_bound,
                               self.best_int,
                               self.initial_optimality_gap,
                               self.optimality_gap])
                               
        return(tree_stats)

    def get_node_stats(self, node_key):
        """
        Returns node statistics

        Returns:
            numpy.ndarray: Node specific statistics.
                - length of zub
                - legth of zlb
                - node specific primal value
                - node depth level
                - legth of support
                - whether the node has the lower bound
                - whether the node has the upper bound
        """
        # Returns summary stats for a given node
        node = self.all_nodes[node_key]
        len_support = len(node.support) if node.support else 0
        has_lb = (self.lower_bound == node.primal_value)
        has_ub = (self.best_int == node.upper_bound)
        node_stats = np.array([len(node.zub),
                               len(node.zlb),
                               node.primal_value,
                               node.level,
                               len_support,
                               has_lb,
                               has_ub])
        return(node_stats)

    def get_var_stats(self, node_key, j):
        """
        Returns variable statistics

        Returns:
            numpy.ndarray: Node and Variable specific statistics.
                - primal beta
                - z value
                - upper z value 
        """
        node = self.all_nodes[node_key]

        # Case when node has no support (used for retrobranching)
        if len(node.support) == 0: 
            print('No Support found for Node during Retrobranching')
            # Values chosen to show lack of information for agent
            return np.array([0, -1, 0])
        
        # Returns summary stats for branching on j in a node
        index = [i for i in range(len(node.support)) if node.support[i] == j][0]
        var_stats = np.array([node.primal_beta[index],
                          node.z[index], node.upper_z[index]])
        
        return(var_stats)
    
    def get_state(self, node_key, j):
        '''
        Returns numpy array of 34 values containing problem and variable static states
        as well as tree, node, and variable current states.
        '''
        # Concatenates overall state for possible branch
        state = np.concatenate((self.problem.prob_stats,
                  self.problem.var_stats[j,:],
                  self.tree_stats,
                  self.get_node_stats(node_key),
                  self.get_var_stats(node_key, j)))
        return(state)

    def max_frac_branch(self):
        # Finds node with greatest fractional part
        best_node_key = None
        best_frac = 0
        best_index = None
        for node_key in self.active_nodes:
            node = self.active_nodes[node_key]
            z = node.z
            support = node.support
            diff = [min(1-z[i],z[i]) for i in range(len(support))]
            max_diff = max(diff)
            potential_j = [i for i in range(len(support)) if diff[i]== max_diff][0]
            if max_diff > best_frac:
                best_node_key = node_key
                best_frac = max_diff
                best_index = support[potential_j]
        return(best_node_key, best_index)

    def int_sol(self, node):
        # Check if within tolerance to an integer solution
        for i in node.support:
            if i not in node.zlb and i not in node.zub:
                # beta_i = node.primal_beta[node.support.index(i)]
                z_i = node.z[node.support.index(i)]
                residual = min(z_i, 1-z_i)
                if residual > self.int_tol:
                    return(False)
        return(True)

    def prune(self, node_keys):
        # Iterate through node keys to find those that aren't active
        for node_key in node_keys:
            if self.active_nodes[node_key].primal_value > self.best_int:
                del self.active_nodes[node_key]

    def update_lower_bound(self):
        # Update the best lower bound over all active nodes
        if len(self.active_nodes) == 0:
            self.lower_bound = self.best_int
        else:
            self.lower_bound = \
                             min(self.active_nodes.values(), \
                                 key=attrgetter('primal_value')).primal_value
            self.lower_bound_node_key = reverse_lookup(self.active_nodes, \
                                                       min(self.active_nodes.values(), \
                                                           key=attrgetter('primal_value')))
    def solve_node(self, node_key):
        # Solves a node with CD and updates upper bound 
        curr_node = self.active_nodes[node_key]
        ## Call lower solve from problem on current node
        self.problem.lower_solve(curr_node)
        # Update upper bound by rounding soln
        curr_upper_bound = self.problem.upper_solve(curr_node)
        if curr_upper_bound < self.best_int:
            self.best_int = curr_upper_bound
            self.candidate_sol = curr_node.upper_z
            upper_bound_updated = True

        # Check if an integer solution
        if self.int_sol(curr_node):
            if curr_node.primal_value < self.best_int:
                self.best_int = curr_node.primal_value
                self.candidate_sol = curr_node.primal_beta
            del self.active_nodes[node_key]

    def step(self, branch_node_key, j):
        # Add State to Node in Tree
        branch_node = self.active_nodes[branch_node_key]
        # branch_node.state = self.get_state(branch_node_key, j)

        # Branches for beta_j in node branch_node_key and returns True if done
        self.step_counter += 1

        # Create two child nodes
        branch_node = self.active_nodes[branch_node_key]
        new_zlb = branch_node.zlb+[j]
        new_zub = branch_node.zub+[j]
        
        # Branch to beta_j to be 0
        node_name_1 = f'node_{self.node_counter}'
        self.active_nodes[node_name_1] = Node(parent=branch_node, node_key=node_name_1, zlb=new_zlb, zub=branch_node.zub, model = self.problem.model)
        self.node_counter += 1

        # Branch to beta_j to be 1
        node_name_2 = f'node_{self.node_counter}'
        self.active_nodes[node_name_2] = Node(parent=branch_node, node_key=node_name_2, zlb=branch_node.zlb, zub=new_zub, model = self.problem.model)
        self.node_counter += 1

        # Store New Nodes in all_nodes Dictionary
        self.all_nodes[node_name_1] = self.active_nodes[node_name_1]
        self.all_nodes[node_name_2] = self.active_nodes[node_name_2]

        # Store Child Nodes in Tree
        branch_node.assign_children(self.active_nodes[node_name_1], self.active_nodes[node_name_2])
        del self.active_nodes[branch_node_key] # Delete Parent from Active Nodes

        # Solve relaxations in new nodes
        past_upper = self.best_int
        self.solve_node(node_name_1)
        self.solve_node(node_name_2)

        # Check for node eliminations based on upper bound
        if (past_upper > self.best_int):
            self.prune(list(self.active_nodes.keys()))
            
        # Update lower_bound and lower_bound_node_key
        self.update_lower_bound()

        # Update optimality gap
        old_gap = self.optimality_gap
        self.optimality_gap = (self.best_int - self.lower_bound) / self.lower_bound

        # Update stats
        # self.tree_stats = self.get_tree_stats()

        # Return True if solved or within tolerance and False otherwise
        if (len(self.active_nodes) == 0) or (self.optimality_gap <= self.gap_tol):
            return(True, old_gap, self.optimality_gap)
        
        return(False, old_gap, self.optimality_gap)

    def branch_and_bound(self, branch="max"):
        self.start_root(None)

        fin_solving = False
        iters = 0
        num_pos = 0
        while (not fin_solving) and (iters < MAX_ITERS):
            node_key, j = self.max_frac_branch()
            # Take a step
            fin_solving, old_gap, new_gap = self.step(node_key, j)

            # Update iterations and number positive
            if old_gap > new_gap:
                num_pos += 1
            iters += 1
        
        reward = -iters + 1

        return(iters, reward, len(self.candidate_sol), self.optimality_gap)

    # The get_state_pairs function can remain unchanged if it's still required.

    def get_state_pairs(self, node):
        '''
        Recursively collect tree edges, parent and child states pairs

        input:
            node: node of tree
        return:
            list of (previous state, state, reward) tuples
        '''
        pairs = []

        if not node:
            return pairs

        # Check left child
        if node.left:
            if node.left.is_leaf:
                # If child is leaf reward is 0
                pairs.append((node.state, node.left.state, 0))
            else:
                pairs.append((node.state, node.left.state, -1))
            pairs.extend(self.get_state_pairs(node.left))

        # Check right child
        if node.right:
            if node.right.is_leaf:
                # If child is leaf reward is -1
                pairs.append((node.state, node.right.state, 0))
            else:
                pairs.append((node.state, node.right.state, -1))
            pairs.extend(self.get_state_pairs(node.right))

        return pairs

        


In [49]:
newProblem = Problem(x)
newTree = tree(newProblem)

newTree.start_root()

newTree.step('root_node', 4)
print(newTree.all_nodes)

TypeError: 'NoneType' object is not iterable